In [15]:
from pymongo import MongoClient
import pandas as pd
import src.apis as sr 

In [16]:
client = MongoClient("localhost:27017")
db = client.get_database("prueba")
c= db.get_collection("companies")

One of the objectives of this project is to choose three cities in which there are companies dedicated to the technology sector and which are small or medium-sized companies like ours. The first filter we will apply then will be the "web" category in order to stay with companies that are dedicated to the "Tech" sector. Next, we will choose some cosmopolitan European cities (that facilitate the travels of Account Managers) and of different sizes (1.5M Munich, 5M Barcelona, 10M Paris) to make a sample according to the size of each city.

## Filtros en Mongo

### Paris

In [3]:
proj = {"_id": 0, "name": 1,"offices.country_code":1, "offices.city":1, "number_of_employees": 1, "offices.latitude":1, "offices.longitude":1}
fra = list(c.find({"$and": [{"category_code": "web"}, {"offices.country_code": "FRA"}, {"number_of_employees": {"$lt":100}},
                      {"founded_year": {"$gt": 2000}}]}, proj))
df = pd.DataFrame(fra)
df["offices"] = df.offices.astype(str)
df = df[df["offices"].str.contains('Paris')]


### Barcelona

In [4]:
spa= list(c.find({"$and": [{"category_code": "web"}, {"offices.country_code": "ESP"}, {"number_of_employees": {"$lt":100}},
                      {"founded_year": {"$gt": 2000}}]}, proj))
df2 = pd.DataFrame(spa)
df2["offices"] = df2.offices.astype(str)
df2 = df2[df2["offices"].str.contains('Barcelona')]

### Munich

In [17]:
deu= list(c.find({"$and": [{"category_code": "web"}, {"offices.country_code": "DEU"}, {"number_of_employees": {"$lt":100}},
                      {"founded_year": {"$gt": 2000}}]}, proj))
df3 = pd.DataFrame(deu)
df3["offices"] = df3.offices.astype(str)
df3 = df3[df3["offices"].str.contains('Munich')]

,name,number_of_employees,offices
3,amiando,76,"[{'city': 'Munich', 'country_code': 'DEU', 'la..."
15,be2,2,"[{'city': 'Munich', 'country_code': 'DEU', 'la..."


## Búsqueda de empresas similares en cada ciudad

Nos damos cuenta de que existen algunas empresas Tech(web) en las ciudades en las que hemos buscado y nos quedamos con la selección de ciudades que hemos realizado. También procedemos a crear un dataframe sucio con algunos de los datos que nos pudiesen llegar a interesar sobre las queries a MongoDB. Concatenamos los datos de las queries obtenidos y creamos un dataframe con aquellas empresar de tamaño mediano (+10 trabajadores) por cada país. 
    

In [6]:
df_final=pd.concat([df, df2, df3])

In [7]:
df_final.drop(df_final[df_final.number_of_employees < 10].index, inplace=True)

In [8]:
df_final['city'], df_final['countrycode'],df_final['latitude'],df_final['longitude'],df_final['else']  = df_final['offices'].str.split(',', 4).str

<ipython-input-8-23573977b66e>:1: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  df_final['city'], df_final['countrycode'],df_final['latitude'],df_final['longitude'],df_final['else']  = df_final['offices'].str.split(',', 4).str


In [9]:
df_final

,name,number_of_employees,offices,city,countrycode,latitude,longitude,else
0,Netvibes,35,"[{'city': 'Paris', 'country_code': 'FRA', 'lat...",[{'city': 'Paris','country_code': 'FRA','latitude': 48.870806,'longitude': 2.34668},"{'city': 'San Francisco', 'country_code': 'US..."
8,amiando,76,"[{'city': 'Munich', 'country_code': 'DEU', 'la...",[{'city': 'Munich','country_code': 'DEU','latitude': 48.124827,'longitude': 11.556452},"{'city': 'London', 'country_code': 'GBR', 'la..."
13,Zefanclub,15,"[{'city': 'Paris', 'country_code': 'FRA', 'lat...",[{'city': 'Paris','country_code': 'FRA','latitude': 48.868478,'longitude': 2.339843},"{'city': 'Istanbul', 'country_code': 'TUR', '..."
23,Calameo,10,"[{'city': 'Paris', 'country_code': 'FRA', 'lat...",[{'city': 'Paris','country_code': 'FRA','latitude': 48.871174,'longitude': 2.308019}],NaN
26,Stupeflix,15,"[{'city': 'Paris', 'country_code': 'FRA', 'lat...",[{'city': 'Paris','country_code': 'FRA','latitude': 48.8785618,'longitude': 2.3603689},"{'city': 'San Francisco', 'country_code': 'US..."
32,Webedia,50,"[{'city': 'Paris', 'country_code': 'FRA', 'lat...",[{'city': 'Paris','country_code': 'FRA','latitude': 48.856614,'longitude': 2.3522219}],NaN
33,Webzzle,10,"[{'city': 'Palo Alto', 'country_code': 'USA', ...",[{'city': 'Palo Alto','country_code': 'USA','latitude': 37.4417,'longitude': -122.1417},"{'city': 'Paris', 'country_code': 'FRA', 'lat..."
42,Ykone,10,"[{'city': 'Paris', 'country_code': 'FRA', 'lat...",[{'city': 'Paris','country_code': 'FRA','latitude': 48.8566667,'longitude': 2.3509871}],NaN
0,Splendia,70,"[{'city': 'Barcelona', 'country_code': 'ESP', ...",[{'city': 'Barcelona','country_code': 'ESP','latitude': None,'longitude': None}],NaN
23,Teambox,35,"[{'city': 'Barcelona', 'country_code': 'ESP', ...",[{'city': 'Barcelona','country_code': 'ESP','latitude': 41.4063346,'longitude': 2.1920231},"{'city': 'Redwood City', 'country_code': 'USA..."
